# KoChatGPT SFT+RLHF 실습 프로젝트

#### __할 일__
  - Back bone 모델 교체;  
    - SFT모델, 리워드 모델, actor/critic모델  
  - 

__한 일__
  - generator선언 시 'pipeline'클래스의 초기화 인자 'model' 세팅


__결과__
  - SFT에 들어가는 QnA 입력 형태가 이전 GPT chatbot의 경우와 다르다.  
    타겟 데이터에 Q부분을 마스킹 처리하는 것.
  - 

In [2]:
import os
import json
import copy
from copy import deepcopy
import logging
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments

from datasets import load_dataset
from dataclasses import dataclass

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "skt/kogpt2-base-v2"
# model_name = "skt/ko-gpt-trinity-1.2B-v0.5"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

#### 모델 불러오기

In [4]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2').to(device)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    # 특수 토큰들을 '</s>'로 통일한 이유가 있는 걸까?
    # 설정을 바꾸면 vocab크기가 달라져 오류 발생; CUBLAS_STATUS_NOT_INITIALIZED
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    # 'skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', unk_token='<u>', pad_token='<p>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


In [6]:
print(model.config)
tot_numt_param = 0
for layer_name, params in model.named_parameters():
    print(layer_name,'---', params.shape)
    n_p = 1
    for p in params.shape:
        n_p *= p
    tot_numt_param += n_p
    
print("tot_numt_param: ", tot_numt_param)

GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
  

#### 데이터셋 불러오기 (SFT)

In [5]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [6]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [7]:
train_dataset = SFT_dataset(data_path_1_SFT='/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [8]:
# train_dataset.input_ids[0]를 디코딩해보세요.
print(tokenizer.decode(train_dataset.input_ids[0]))

### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s>


#### 트레이너 세팅 (SFT)

In [14]:
run_idx = 0

training_args = TrainingArguments(
    output_dir="/aiffel/aiffel/KoChatGPT/sft_ckpts",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

#### 훈련 (SFT)

In [15]:
trainer.train()
# RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`
model.save_pretrained('/aiffel/aiffel/KoChatGPT/output_1_SFT_{:03d}'.format(run_idx))

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.007000
1000,2.172100
1500,2.510200


#### 모델 출력 예제 (SFT)

In [ ]:
# 저장 된 모델이 아니라 현제 로드된 모델 사용해보기
# generator = pipeline('text-generation', model='/aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

In [16]:
list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?',
    """\
    Q: 로저는 5개의 테니스 공이 있습니다. 그는 테니스 공이 들어간 두 개의 캔을 샀고, 각각의 캔에는 3개의 공이 들어있습니다. 이때 테니스 공의 총 개수는?
    A: 로저는 5개의 공으로 시작했고, 2개의 캔에는 각각 3개의 공이 들어있으니 총 6개의 공이 있습니다. 그러므로 5 + 6 = 11 입니다.
    Q: 카페에는 23개의 사과가 있습니다. 20개는 점심에 먹었했고, 6개를 추가로 샀습니다. 이때 사과의 총 개수는?
    A:""",
    """\
    SYSTEM: 당신은 화장품회사의 고객관리부 팀장입니다.
    Q: 스킨케어 브랜드와 가치를 소개하는 따뜻하고 친근한 신규 구독자 환영 이메일을 작성하세요.
    A:""",
    """\
    아래 문맥에 따라 질문에 답하세요. 답변은 짧고 간결하게 작성하세요. 답변이 확실하지 않은 경우 \"답변이 확실하지 않음\"으로 응답하세요.
    맥락: Teplizumab은 뉴저지의 제약 회사인 Ortho Pharmaceutical 제약에서 유래되었습니다. 거기에서 과학자들은 OKT3라는 이름의 초기 버전 항체를 개발했습니다. 원래 쥐로부터 유래된 이 분자는 T 세포의 표면에 결합하여 그들의 세포 죽음을 막을 수 있었습니다. 1986년에는 신장 이식 후 장기 거부반응을 예방하는 데 도움이 되도록 승인되어 인간이 사용하도록 허용된 최초의 치료 항체가 되었습니다.
    질문: OKT3는 어디에서 유래되었나요?
    답변:""",]

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'불고기용 고기는 불고기, 돼지고기, 소고기, 양고기 등 다양한 종류가 있습니다. 일반적으로 불고기용 고기의 경우, 불고기와 돼지고기를 함께 먹을 수 있는 경우가 많습니다. 따라서 불고기용은 불고기보다 비싸지만, 불고기는 불고기보다 더 맛있게 먹을 수 있습니다.\n\

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 46대 부통령을 수행한 년도가 없습니다. 따라서 정확한 답변을 제공할 수 없습니다. "리처드 닉슨"이 무엇을 의미하는지 더 자세한 정보를 제공해주시면 더 정확한 답변을 드릴 수 있습니다. 리처드 닉슨은 미국 역사상 가장 유명한 부통령 중 한 명입니다. 리처드 닉슨의

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오 헤어 국제공항은 미국 캘리포니아 주에 위치해 있습니다. Sicraelele International Airports 소속 항공사입니다. Sichelor International AI language model and Images of translation. Sichelle International information

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇이므로 미세먼지 여부를 알 수 없습니다. 하지만, 미세먼지 농도는 보통 매우 높기 때문에 대기 질이 좋지 않을 것으로 예상됩니다. 따라서 외출 시 마스크를 착용하시는 것이 좋습니다.\n\n미세먼지 농도가 높아지면 건강에 해로울 수


In [18]:
torch.cuda.empty_cache()

### 리워드 모델 훈련

In [26]:
from typing import Optional

# 아래 모듈은 홈 디렉토리에서 임포트 해야한다
cwd = os.getcwd()
os.chdir(os.getenv("HOME"))
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
os.chdir(cwd)

from transformers import AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

import loralib as lora

#### 모델 구성

In [27]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained

    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [28]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

# 아래 명령어의 옵션이 주는 효과?; lora_rank=0 이건 효과가 있는 것인가?
with NaiveStrategy().model_init_context():
    model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### 입력 데이터셋 구성

In [45]:
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)

    total_data_ranking2chosen.extend(one_data_ranking2chosen)

In [46]:
print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [49]:
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '해마가 위치 정보를 파악하는 역할을 하는 것은 해마에 무엇이 존재하기 때문인가?', 'chosen': '해마는 해마 측근 관련 뉴런이 존재하여 위치 정보를 파악하고 기억하는 역할을 합니다. 해마는 고양이, 쥐, 사람 등 다양한 동물들의 뇌에서 발견되며, 오랜 시간 동안 연구된 결과, 해마는 공간 정보 처리를 담당하는 뇌의 중요한 영역 중 하나로 알려져 있습니다.', 'rejected': '위치 정보는 국가들의 페이지를 통한 국가들의 페이지를 통한 국가들의 페이지 국가들의 페이지 국가들의 페이지 국가들의 페이지 국가들의 페이지 국가들의 페이지 국가들의 페이지 국가들의 페이지 국가들의 페이지 국가들의 페이지 국가들의'}


In [31]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print("total_data_ranking2chosen: ", len(total_data_ranking2chosen))
print("train_data: ", len(train_data))
print("eval_data: ", len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

total_data_ranking2chosen:  30660
train_data:  1000
eval_data:  200


100%|██████████| 200/200 [00:00<00:00, 1091.50it/s]


In [33]:
idx = 1
print('-'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('-'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('-'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

----------------------------------------------------------------------
## prompt ##
흑고래의 무게는 어느 정도야
----------------------------------------------------------------------
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
----------------------------------------------------------------------
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


#### 트레이너 구성 및 모델 훈련

In [34]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

In [51]:
# LoRA 옵션의 의미?
# trainer.fit(use_lora=0)

# 왜 진행상황 막대가 제대로 출력되지 않을까?
# model.save_pretrained('aiffel/KoChatGPT/output_2_RM')
model.save_pretrained('/aiffel/aiffel/KoChatGPT/output_2_RM_{:03d}'.format(run_idx))

In [37]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: 0.3


In [38]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.
reward score: -0.4


In [39]:
input_text = "인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다.
reward score: -0.5


In [40]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: -0.4


In [50]:
torch.cuda.empty_cache()

### PPO 모델 훈련

In [42]:
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer

# 아래 Strategy 말고 다른 것이 있을까?
# from chatgpt.trainer.strategies import NaiveStrategy


In [52]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='/aiffel/aiffel/KoChatGPT/output_1_SFT_{:03d}'.format(0),
                                     lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='/aiffel/aiffel/KoChatGPT/output_2_RM_{:03d}'.format(0),
                                       lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [53]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [54]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [55]:
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [56]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [57]:
len(list_prompt)

12000

In [60]:
list_prompt[0]

'번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'

In [58]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [61]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

# model.save_pretrained('aiffel/KoChatGPT/output_3_PPO')
model.save_pretrained('/aiffel/aiffel/KoChatGPT/output_3_PPO_{:03d}'.format(run_idx))

Episode [10/10]: 100%|██████████| 3/3 [00:19<00:00,  6.37s/it]


In [62]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?',
    """\
    Q: 로저는 5개의 테니스 공이 있습니다. 그는 테니스 공이 들어간 두 개의 캔을 샀고, 각각의 캔에는 3개의 공이 들어있습니다. 이때 테니스 공의 총 개수는?
    A: 로저는 5개의 공으로 시작했고, 2개의 캔에는 각각 3개의 공이 들어있으니 총 6개의 공이 있습니다. 그러므로 5 + 6 = 11 입니다.
    Q: 카페에는 23개의 사과가 있습니다. 20개는 점심에 먹었했고, 6개를 추가로 샀습니다. 이때 사과의 총 개수는?
    A:""",
    """\
    SYSTEM: 당신은 화장품회사의 고객관리부 팀장입니다.
    Q: 스킨케어 브랜드와 가치를 소개하는 따뜻하고 친근한 신규 구독자 환영 이메일을 작성하세요.
    A:""",
    """\
    아래 문맥에 따라 질문에 답하세요. 답변은 짧고 간결하게 작성하세요. 답변이 확실하지 않은 경우 \"답변이 확실하지 않음\"으로 응답하세요.
    맥락: Teplizumab은 뉴저지의 제약 회사인 Ortho Pharmaceutical 제약에서 유래되었습니다. 거기에서 과학자들은 OKT3라는 이름의 초기 버전 항체를 개발했습니다. 원래 쥐로부터 유래된 이 분자는 T 세포의 표면에 결합하여 그들의 세포 죽음을 막을 수 있었습니다. 1986년에는 신장 이식 후 장기 거부반응을 예방하는 데 도움이 되도록 승인되어 인간이 사용하도록 허용된 최초의 치료 항체가 되었습니다.
    질문: OKT3는 어디에서 유래되었나요?
    답변:""",]

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'네, 불고기를 파는 상점에서 불고기용 고기인 '불고기용 고기(bangtwoak)'를 판매하는 경우가 많습니다. 불고기용 고기는 주로 식초, 소스, 된장, 참기름 등 다양한 재료가 사용되며, 일반적으로 불고기 양념에 불고기를 넣은 소스, 무 등을 넣어서 만든 양념고기와 된장으로도 많이 사용되는 것이 특징입니다. 高木洞山 (불고기를 판매하는 상점의 이름)\n\n불고기용 고기는 주로 육류, 고기, 소스 등으로 조리됩니다. 불고기용 고기는 일반적으로 돼지고기를 요리하여 제조됩니다. 불고기의 기본 재료 중 하나로 알려진 것으로 알려져 있으며, 불고기용 고기는 고기를 구워내는데 필수적인 것으로 알려져 있습니다.神陽 洪積土 (불고기용의 고기)\n\n불고기용은 일반적으로 고기 한우(불고기용 고기를 썰어내 양념한 고기를 사용하여 만들어집니다. 불고기용은 주로 육류, 소, 양 등으로 제조됩니다. 불고기용은 일반적으로 육류, 고기, 생선류 등으로 조리됩니다. 洪積土 

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 43대 부통령직을 수행한 년도는 정확히 밝혀지지 않았습니다. 承法\n tissue of v\n- \n- 닉슨이 47대 부통령직을 맡은 정확한 정보는 알려지지 않았습니다. tion\n- 닉슨은 34대 부통령직을 맡은 정확한 연도는 알려지지 않았습니다. \n- 닉슨은 39대 부통령직을 맡았던 정확한 연도는 공개되지 않았습니다. \n- 닉슨은 36대 부통령직을 맡았던 정확한 연도는 알려지지 않았습니다. \n- 닉슨은 39대 부통령직을 맡은 정확한 연도는 알려지지 않았습니다. \n\n리처드 닉슨은 41대 부통령직을 맡았던 정확한 연도는 알려져 있지 않습니다. tion\n- 닉슨은 41대 부통령직을 수행한 구체적인 연도는 밝혀지지 않았지만, 닉슨은 39대 부통령직에 있었던 정확한 연도는 아직

### Back-bone 모델 교체

In [1]:
import os
import json
import copy
from copy import deepcopy
import logging
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam

import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments

from datasets import load_dataset
from dataclasses import dataclass

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# model_name = "skt/kogpt2-base-v2"
model_name = "skt/ko-gpt-trinity-1.2B-v0.5"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

#### 모델 불러오기

In [3]:
# from transformers import AutoConfig
ko_gpt_config = AutoConfig.from_pretrained(model_name,
#                                            ignore_mismatched_sizes=True,
# **{"vocab_size": 32000,
#   "ignore_mismatched_sizes" :True  }
)  

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_name, config=ko_gpt_config)#.to(device)
# model.resize_token_embeddings(len(tokenizer))

In [5]:
print(model.config)
tot_numt_param = 0
for layer_name, params in model.named_parameters():
    print(layer_name,'---', params.shape)
    n_p = 1
    for p in params.shape:
        n_p *= p
    tot_numt_param += n_p
    
print("tot_numt_param: ", tot_numt_param)

GPT2Config {
  "_name_or_path": "skt/ko-gpt-trinity-1.2B-v0.5",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 8,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1920,
  "n_head": 16,
  "n_inner": 7680,
  "n_layer": 24,
  "n_positions": 1024,
  "pad_token_id": 8,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.28.0",
  "use_cache": true,
  "vocab_size": 51200
}

transformer.wte.weight --- torch.Size([51200, 1920])
transformer.wpe.weight --- torch.Size([1024, 1920])
transformer.h.0.ln_1.weight --- torch.Size(

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    # 특수 토큰들을 '</s>'로 통일한 이유가 있는 걸까?
    # 설정을 바꾸면 vocab크기가 달라져 오류 발생; CUBLAS_STATUS_NOT_INITIALIZED
    # model_name, bos_token='<bos>', eos_token='<eos>', unk_token='<unk>', pad_token='<pad>',
    model_name, bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=128,
#     **{"vocab_size": 32000}
)

print(tokenizer)

In [ ]:
len(tokenizer)
model.resize_token_embeddings(len(tokenizer))

#### 데이터셋 불러오기 (SFT)

In [ ]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [ ]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [ ]:
train_dataset = SFT_dataset(data_path_1_SFT='/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

In [ ]:
# train_dataset.input_ids[0]를 디코딩해보세요.
print(tokenizer.decode(train_dataset.input_ids[0]))
len(train_dataset.input_ids[0])

#### 트레이너 세팅 (SFT)

In [ ]:
run_idx = 1

training_args = TrainingArguments(
    output_dir="/aiffel/aiffel/KoChatGPT/sft_ckpts_{:03d}".format(run_idx),
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True,
    group_by_length = True, # Bucketing을 적용
    abc
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

#### 훈련 (SFT)

In [ ]:
trainer.train()
# RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`
model.save_pretrained('/aiffel/aiffel/KoChatGPT/output_1_SFT_{:03d}'.format(run_idx))

#### 모델 출력 예제 (SFT)

In [ ]:
# 저장 된 모델이 아니라 현제 로드된 모델 사용해보기
# generator = pipeline('text-generation', model='/aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

In [16]:
list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?',
    """\
    Q: 로저는 5개의 테니스 공이 있습니다. 그는 테니스 공이 들어간 두 개의 캔을 샀고, 각각의 캔에는 3개의 공이 들어있습니다. 이때 테니스 공의 총 개수는?
    A: 로저는 5개의 공으로 시작했고, 2개의 캔에는 각각 3개의 공이 들어있으니 총 6개의 공이 있습니다. 그러므로 5 + 6 = 11 입니다.
    Q: 카페에는 23개의 사과가 있습니다. 20개는 점심에 먹었했고, 6개를 추가로 샀습니다. 이때 사과의 총 개수는?
    A:""",
    """\
    SYSTEM: 당신은 화장품회사의 고객관리부 팀장입니다.
    Q: 스킨케어 브랜드와 가치를 소개하는 따뜻하고 친근한 신규 구독자 환영 이메일을 작성하세요.
    A:""",
    """\
    아래 문맥에 따라 질문에 답하세요. 답변은 짧고 간결하게 작성하세요. 답변이 확실하지 않은 경우 \"답변이 확실하지 않음\"으로 응답하세요.
    맥락: Teplizumab은 뉴저지의 제약 회사인 Ortho Pharmaceutical 제약에서 유래되었습니다. 거기에서 과학자들은 OKT3라는 이름의 초기 버전 항체를 개발했습니다. 원래 쥐로부터 유래된 이 분자는 T 세포의 표면에 결합하여 그들의 세포 죽음을 막을 수 있었습니다. 1986년에는 신장 이식 후 장기 거부반응을 예방하는 데 도움이 되도록 승인되어 인간이 사용하도록 허용된 최초의 치료 항체가 되었습니다.
    질문: OKT3는 어디에서 유래되었나요?
    답변:""",]

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'불고기용 고기는 불고기, 돼지고기, 소고기, 양고기 등 다양한 종류가 있습니다. 일반적으로 불고기용 고기의 경우, 불고기와 돼지고기를 함께 먹을 수 있는 경우가 많습니다. 따라서 불고기용은 불고기보다 비싸지만, 불고기는 불고기보다 더 맛있게 먹을 수 있습니다.\n\

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 46대 부통령을 수행한 년도가 없습니다. 따라서 정확한 답변을 제공할 수 없습니다. "리처드 닉슨"이 무엇을 의미하는지 더 자세한 정보를 제공해주시면 더 정확한 답변을 드릴 수 있습니다. 리처드 닉슨은 미국 역사상 가장 유명한 부통령 중 한 명입니다. 리처드 닉슨의

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오 헤어 국제공항은 미국 캘리포니아 주에 위치해 있습니다. Sicraelele International Airports 소속 항공사입니다. Sichelor International AI language model and Images of translation. Sichelle International information

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇이므로 미세먼지 여부를 알 수 없습니다. 하지만, 미세먼지 농도는 보통 매우 높기 때문에 대기 질이 좋지 않을 것으로 예상됩니다. 따라서 외출 시 마스크를 착용하시는 것이 좋습니다.\n\n미세먼지 농도가 높아지면 건강에 해로울 수
